In [55]:
class Queue:
    queue = []
    
    def put(self,val):
        self.queue.append(val)

    def get(self):
        return self.queue.pop(0)
    
    def __str__(self):
        return str(self.queue)

class Grid:
    def __init__(self, width, height):
        self.grid = [[0 for x in range(width)] for y in range(height)]
        self.width = width
        self.height = height
    
    def __str__(self):
        grid_string = ""
        for row in self.grid:
            row_str = ""
            for val in row:
                row_str += str(val)
            grid_string += row_str + "\n"
        return grid_string
     
    """
    Gets the value at the given coordinate
    """
    def get_val(self,x,y):
        try:
            return self.grid[y][x]
        except:
            return None
    
    """
    Get the coordinates which are valid neighbours
    A valid neighbour is a orthogonal coordinate which is an open space
    """
    def get_neighbours(self,x,y):
        valid = []
        directions = [[1,0],[-1,0],[0,1],[0,-1]]
        for direction in directions:
            if self.get_val(x+direction[0], y+direction[1]) == ".":
                valid.append([x+direction[0], y+direction[1]])
        return valid
        
    """
    Generate the map in accordance to the given algorithm
    """
    def generate_map(self, key):
        for y in range(height):
            for x in range(width):
                self.grid[y][x] = "." if self.__is_open_space(x,y,key) else "#"
    
    """
    If the value at the given grid coordinate is an open space, return True.
    Otherwize return False
    """
    def __is_open_space(self,x,y,key):
        val1 = x*x + 3*x + 2*x*y + y + y*y
        val1 += key
        binary_val = bin(val1)[2:]
        nr_ones = 0
        for x in binary_val:
            if x == "1":
                nr_ones += 1
        if nr_ones % 2 == 0:
            return True
        else:
            return False

### Implement BFS - Breath First Search
Keep track of the frontier! I.e the nodes we want to visit.  

For each node we visit, find its valid neighbors.
If we haven't visited the neighbor before, add it to our frontier queue and mark that we reached that frontier node from the one we're currently on.

In [56]:
frontier = Queue()

goal_x = 31
goal_y = 39

width = 80
height = 45
grid = Grid(width, height)
grid.generate_map(1352)
print(grid.get_neighbours(1,2))
print(grid)

[[2, 2], [0, 2], [1, 1]]
.##.##............###.....#####.#####....#..##..#.###....#....#......#..#.##.#..
#.######.##.####...#.##....#..#.#..#.##.####.##....####.###.#.##.##.###......###
....##.#..#.##.#.#.######..#.##..#.####..#.#######..###..##..#.##.#.##########.#
##....###......#..#..#..####.###..#..#.#.##..#.......###...#....#....##...#.....
.####.####..###.#.##.#.#...#..#.#.##.##......#..####..#.######.#####....#...##..
.......######.###..###..##.#..###..##.#..##.####...#..##....##.#...#.###.######.
..##.#..##..#.......###.#..###......#.#####..#.#...#...#..#....#...#...###....#.
##.#........##.##.#.....#....#.##.#.###..#.#.####..##.###########..##.#....##...
.##.######.#.#.##...###.##...##.#..#..##.##...#######.....##..#######.#########.
#.##....##..##.#.###..#.###.#.##.#..#..##.#.....#..#####....#..##..##...#..##.#.
##.#..#.###.#..#...#.##...#..#.#..#.......##..#.##..##.#.##.....#.....#..#...###
.######..#..#.######.#####.#..###..##.###.####...#.....#.#.##.#.##.#####...#.###
...